In [1]:
import pandas as pd
import sys
import os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from pocketml.preprocessing.split import train_test_split
from pocketml.anomaly.gaussian_anomaly import GaussianAnomalyDetector
from pocketml.metrics.classification import classification_report

# load creditcard.csv
df = pd.read_csv("datasets/creditcard.csv")
X = df.drop(["Time","Class"], axis=1).values
y = df["Class"].values

# split normals and anomalies
X_norm = X[y==0]
X_ano  = X[y==1]
y_norm = y[y==0]
y_ano  = y[y==1]

In [3]:
# train/test split on normals only
Xn_tr, Xn_te, _, _ = train_test_split(X_norm, y_norm,
                                      test_size=0.2, random_state=0)
# build test set = held-out normals + all anomalies
X_test = np.vstack([Xn_te, X_ano])
y_test = np.hstack([np.zeros(len(Xn_te)), np.ones(len(X_ano))])

# fit detector and predict
detector = GaussianAnomalyDetector(percentile=1).fit(Xn_tr)
y_pred   = detector.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["normal","fraud"]))

              precision    recall  f1-score   support

      normal       1.00      0.99      0.99     56863
       fraud       0.34      0.63      0.44       492

    accuracy                           0.99     57355
   macro avg       0.67      0.81      0.72     57355
weighted avg       0.99      0.99      0.99     57355

